# ALGORITMOS BASADOS EN SERIES TEMPORALES

## 1) Modelo de Promedio Ponderado de Periodos Anteriores (PPPA)
* Toma como base de cálculo el Período Actual de la Ventana de Reposición Definida.
* El mismo período del mes anterior.
* El mismo período del año anterior.
* Con los 3 valores hace un promedio simple ponderado
* Podría agregarse mayor peso a cada uno de los factores si hiciera falta.

### Estee modelo calcula la demanda como un promedio ponderado de:

* **Mes en curso (t)** → Refleja la demanda actual.
* **Mes anterior (t-1)** → Captura tendencias recientes.
* **Mismo mes del año anterior (t-12)** → Captura estacionalidad.

### Fórmula General:
$ 𝐷𝑡=𝛼⋅𝑉𝑡 + 𝛽⋅𝑉𝑡−1 + 𝛾⋅𝑉𝑡−12$

#### Donde:
* Dt = Demanda estimada del mes actual.
* 𝑉𝑡 = Ventas del mes en curso.
* 𝑉𝑡−1 = Ventas del mes anterior.
* 𝑉𝑡−12 = Ventas del mismo mes del año anterior.
* 𝛼,𝛽,𝛾 = Pesos asignados a cada variable, definidos con base en la importancia de cada factor.

### 📌 Este modelo es útil cuando la demanda sigue un patrón estacional anual y presenta tendencias recientes que afectan las ventas.


In [1]:
import pandas as pd
import numpy as np

def forecast_basic(df, current_date, period_length=30):
    """
    Calcula la demanda estimada utilizando el algoritmo básico.

    Parámetros:
    - df: DataFrame de pandas con la información histórica de ventas.
    - current_date: Fecha de referencia para el cálculo.
    - period_length: Número de días que conforman el período (por defecto 30).

    El algoritmo utiliza tres períodos:
      1. Últimos 'period_length' días: desde current_date - (period_length - 1) hasta current_date.
      2. Los 'period_length' días anteriores: desde current_date - (2*period_length - 1) hasta current_date - period_length.
      3. El mismo período del año anterior: desde current_date - 1 año - (period_length - 1) hasta current_date - 1 año.

    La demanda estimada se calcula como el promedio de las ventas en estos tres períodos.
    """
    # Facctores de Pnderación
    # Cada uno de los 3 períodos
   
    factor_last = 70
    factor_previous = 20
    factor_year = 10


    # Definir rangos de fechas para cada período
    last_period_start = current_date - pd.Timedelta(days=period_length - 1)
    last_period_end = current_date

    previous_period_start = current_date - pd.Timedelta(days=2 * period_length - 1)
    previous_period_end = current_date - pd.Timedelta(days=period_length)

    same_period_last_year_start = current_date - pd.DateOffset(years=1) - pd.Timedelta(days=period_length - 1)
    same_period_last_year_end = current_date - pd.DateOffset(years=1)

    actual_period_start = current_date + pd.Timedelta(+ 1)
    actual_period_end = current_date + pd.Timedelta(days=period_length + 2)

    # Filtrar los datos para cada uno de los períodos
    df_last = df[(df['Fecha'] >= last_period_start) & (df['Fecha'] <= last_period_end)]
    df_previous = df[(df['Fecha'] >= previous_period_start) & (df['Fecha'] <= previous_period_end)]
    df_same_year = df[(df['Fecha'] >= same_period_last_year_start) & (df['Fecha'] <= same_period_last_year_end)]
    df_actual = df[(df['Fecha'] >= actual_period_start) & (df['Fecha'] <= actual_period_end)]

    # Agregar las ventas (unidades) por artículo y sucursal para cada período
    sales_last = df_last.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                        .sum().reset_index().rename(columns={'Unidades': 'ventas_last'})
    sales_previous = df_previous.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                                .sum().reset_index().rename(columns={'Unidades': 'ventas_previous'})
    sales_same_year = df_same_year.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                                  .sum().reset_index().rename(columns={'Unidades': 'ventas_same_year'})
    sales_actual = df_actual.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                                .sum().reset_index().rename(columns={'Unidades': 'actual'})

    # Unir la información de los tres períodos
    forecast_df = pd.merge(sales_last, sales_previous, on=['Codigo_Articulo', 'Sucursal'], how='outer')
    forecast_df = pd.merge(forecast_df, sales_same_year, on=['Codigo_Articulo', 'Sucursal'], how='outer')
    forecast_df = pd.merge(forecast_df, sales_actual, on=['Codigo_Articulo', 'Sucursal'], how='outer')
    forecast_df.fillna(0, inplace=True)

    # Calcular la demanda estimada como el promedio de las ventas de los tres períodos
    forecast_df['forecast'] = (forecast_df['ventas_last'] * factor_last +
                               forecast_df['ventas_previous'] * factor_previous +
                               forecast_df['ventas_same_year'] * factor_year) / (factor_year + factor_last + factor_previous)

    # Redondear la predicción al entero más cercano
    forecast_df['forecast'] = forecast_df['forecast'].round().astype(int)

    return forecast_df

def evaluate_forecast(forecast_df, actual_df, merge_keys=['Codigo_Articulo', 'Sucursal']):
    """
    Evalúa la precisión del algoritmo de predicción calculando métricas de error.

    Parámetros:
    - forecast_df: DataFrame con las predicciones (debe incluir la columna 'forecast').
    - actual_df: DataFrame con las ventas reales (debe incluir la columna 'actual').
    - merge_keys: Lista de columnas en las que se realizará la unión de ambos DataFrames.

    Retorna un diccionario con las métricas MAE, MSE y MAPE.
    """
    # Unir las predicciones con los valores reales
    merged = pd.merge(forecast_df, actual_df, on=merge_keys, how='inner')

    # Calcular las métricas de error
    mae = np.mean(np.abs(merged['forecast'] - merged['actual']))
    mse = np.mean((merged['forecast'] - merged['actual'])**2)
    # Se añade un valor pequeño para evitar división por cero
    mape = np.mean(np.abs((merged['forecast'] - merged['actual']) / (merged['actual'] + 1e-9))) * 100

    return {'MAE': mae, 'MSE': mse, 'MAPE': mape}

def get_forecast(df, algorithm='basic', current_date=None, period_length=30):
    """
    Permite la selección del algoritmo de predicción y calcula la demanda estimada.

    Parámetros:
    - df: DataFrame con la información histórica.
    - algorithm: Algoritmo a utilizar (por defecto 'basic').
    - current_date: Fecha de referencia para el cálculo; si es None, se toma la fecha máxima del DataFrame.
    - period_length: Número de días del período a analizar.

    Retorna un DataFrame con las predicciones.
    """
    if current_date is None:
        current_date = df['Fecha'].max()
    else:
        current_date = pd.to_datetime(current_date)
        
    if algorithm == 'basic':
        forecast_df = forecast_basic(df, current_date, period_length)
    else:
        raise ValueError(f"El algoritmo '{algorithm}' no está implementado.")

    return forecast_df


In [2]:
# ELEGIR el PROVEEDOR

proveedor = 327
label = '327-Paladini'

# Cargar Datos
data = pd.read_csv(f'data/{label}.csv')
data.head()

# Adecuar Tipos de Datos
data['Sucursal']= data['Sucursal'].astype(int)
data['Familia']= data['Familia'].astype(int)
data['Rubro']= data['Rubro'].astype(int)
data['SubRubro']= data['SubRubro'].astype(int)
data['Clasificacion']= data['Clasificacion'].astype(int)
data['Codigo_Articulo']= data['Codigo_Articulo'].astype(int)
data['Fecha'] = pd.to_datetime(data['Fecha'])  # Convertir a formato datetime si aún no lo está
data.sort_values(by='Fecha', ascending=True)  # Ordenar por fecha de menor a mayor

# Crear una nueva columna con el mes y el año para análisis temporal
data['Año-Mes'] = data['Fecha'].dt.to_period('M')

# Crear una nueva columna con el formato Año-Semana (AAAA-WW) a partir de la columna Fecha
data['Año-Semana'] = data['Fecha'].dt.strftime('%Y-%W')

# Confirmar que la columna sigue siendo un campo datetime
#print(data.dtypes)

#data = data.sort_values(by='Fecha', ascending=True)  # Ordenar en orden ascendente (del más antiguo al más reciente)
#data = data.reset_index()

# Recortar Cantidad de Datos ULTIMO AÑO COMPLETO
df = data[data['Fecha']>='2021-01-01']

In [3]:
# Calcular la demanda estimada utilizando el algoritmo básico
forecast_result = get_forecast(df, algorithm='basic',current_date='2024-01-01', period_length=15)

# Mostrar un ejemplo de resultados
print("Demanda estimada utilizando el algoritmo básico:")
print(forecast_result.head())

Demanda estimada utilizando el algoritmo básico:
   Codigo_Articulo  Sucursal  ventas_last  ventas_previous  ventas_same_year  \
0              207         1        0.000            0.000             0.746   
1              207        13        0.000            0.000             0.000   
2              207        20        0.000            0.000             1.006   
3              207        25        1.995            0.955             2.755   
4              207        28        3.052            2.002             0.000   

   actual  forecast  
0   0.000         0  
1   2.149         0  
2   1.000         0  
3   2.010         2  
4   1.984         3  


In [4]:
metrics = evaluate_forecast(forecast_result, df)
print("Métricas de evaluación:", metrics)

Métricas de evaluación: {'MAE': np.float64(38.63861060410374), 'MSE': np.float64(8849.247769524769), 'MAPE': np.float64(1152067345000.082)}


In [5]:
current_date ='2024-01-15'
period_length=15


current_date =  pd.to_datetime(current_date)

# Definir rangos de fechas para cada período
last_period_start = current_date - pd.Timedelta(days=period_length - 1)
last_period_end = current_date

previous_period_start = current_date - pd.Timedelta(days=2 * period_length - 1)
previous_period_end = current_date - pd.Timedelta(days=period_length)

same_period_last_year_start = current_date - pd.DateOffset(years=1) - pd.Timedelta(days=period_length - 1)
same_period_last_year_end = current_date - pd.DateOffset(years=1)

actual_period_start = current_date + pd.Timedelta(+ 1)
actual_period_end = current_date + pd.Timedelta(days=period_length + 2)


In [6]:
# Facctores de Pnderación
# Cada uno de los 3 períodos
factor_last = 70
factor_previous = 20
factor_year = 10

 # Filtrar los datos para cada uno de los períodos
df_last = df[(df['Fecha'] >= last_period_start) & (df['Fecha'] <= last_period_end)]
df_previous = df[(df['Fecha'] >= previous_period_start) & (df['Fecha'] <= previous_period_end)]
df_same_year = df[(df['Fecha'] >= same_period_last_year_start) & (df['Fecha'] <= same_period_last_year_end)]
df_actual = df[(df['Fecha'] >= actual_period_start) & (df['Fecha'] <= actual_period_end)]

# Agregar las ventas (unidades) por artículo y sucursal para cada período
sales_last = df_last.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                  .sum().reset_index().rename(columns={'Unidades': 'ventas_last'})
sales_previous = df_previous.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                           .sum().reset_index().rename(columns={'Unidades': 'ventas_previous'})
sales_same_year = df_same_year.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                              .sum().reset_index().rename(columns={'Unidades': 'ventas_same_year'})
sales_actual = df_actual.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                           .sum().reset_index().rename(columns={'Unidades': 'ventas_actual'})

# Unir la información de los tres períodos
forecast_df = pd.merge(sales_last, sales_previous, on=['Codigo_Articulo', 'Sucursal'], how='outer')
forecast_df = pd.merge(forecast_df, sales_same_year, on=['Codigo_Articulo', 'Sucursal'], how='outer')
forecast_df = pd.merge(forecast_df, sales_actual, on=['Codigo_Articulo', 'Sucursal'], how='outer')
forecast_df.fillna(0, inplace=True)

# Calcular la demanda estimada como el promedio de las ventas de los tres períodos
forecast_df['forecast'] = (forecast_df['ventas_last'] * factor_last +
                            forecast_df['ventas_previous'] * factor_previous +
                            forecast_df['ventas_same_year'] * factor_year) / (factor_year + factor_last + factor_previous)

# Redondear la predicción al entero más cercano
forecast_df['forecast'] = forecast_df['forecast'].round().astype(int)


In [7]:
# Mostrar resultado
import ace_tools_open  as tools
tools.display_dataframe_to_user(name="DataFrame Básico", dataframe=forecast_df)

DataFrame Básico


In [ ]:
# Ejemplo de uso del algoritmo de regresión lineal
if __name__ == '__main__':
    # Se asume que se dispone de un DataFrame 'df' con los datos históricos.
    # Por ejemplo, se puede cargar desde un CSV:
    # df = pd.read_csv('datos_ventas.csv', parse_dates=['Fecha'])
    
    # Para efectos ilustrativos se crea un DataFrame con datos simulados.
    fechas = pd.date_range(start='2023-01-01', end='2025-02-10', freq='D')
    np.random.seed(42)
    # data = {
    #     'Fecha': np.random.choice(fechas, 2000),
    #     'Codigo_Articulo': np.random.randint(1000, 1020, size=2000),
    #     'Sucursal': np.random.randint(1, 5, size=2000),
    #     'Precio': np.random.uniform(100, 1000, size=2000),
    #     'Costo': np.random.uniform(80, 900, size=2000),
    #     'Unidades': np.random.randint(1, 10, size=2000),
    #     'Familia': np.random.randint(1, 3, size=2000),
    #     'Rubro': np.random.randint(1, 3, size=2000),
    #     'SubRubro': np.random.randint(1, 3, size=2000),
    #     'Nombre_Articulo': ['Articulo ' + str(i) for i in range(2000, 4000)][:2000],
    #     'Clasificacion': np.random.randint(1, 5, size=2000),
    #     'Año-Mes': pd.PeriodIndex(pd.to_datetime(np.random.choice(fechas, 2000)), freq='M'),
    #     'Año-Semana': ['2024-S' + str(np.random.randint(1, 53)) for _ in range(2000)],
    #     'id_tienda': np.random.randint(1, 5, size=2000),
    #     'formato': np.random.choice(['Formato1', 'Formato2'], size=2000),
    #     'Dia_Semana': np.random.randint(0, 7, size=2000),
    #     'Semana_Año': np.random.randint(1, 53, size=2000)
    # }
    # df = pd.DataFrame(data)
    
    # Se define la fecha de pronóstico: por ejemplo, la última fecha disponible menos period_length
    forecast_date = df['Fecha'].max() - pd.Timedelta(days=30)
    
    # Se ejecuta el algoritmo de regresión lineal para obtener la predicción
    forecast_lr, model, training_data = forecast_linear_regression(
        df, 
        forecast_date=forecast_date, 
        period_length=30
        # Se pueden especificar training_min_date y training_max_date si se desea limitar el conjunto de entrenamiento
    )
    
    print("Demanda estimada utilizando el algoritmo de regresión lineal:")
    print(forecast_lr.head())
    
    # Evaluación del modelo en el conjunto de entrenamiento
    X_train = training_data[['ventas_last', 'ventas_previous', 'ventas_same_year']]
    y_train = training_data['target']
    y_pred_train = model.predict(X_train)
    mae = mean_absolute_error(y_train, y_pred_train)
    mse = mean_squared_error(y_train, y_pred_train)
    print(f"MAE en entrenamiento: {mae:.2f}")
    print(f"MSE en entrenamiento: {mse:.2f}")